In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,2049
2,Huelva,Confirmados PDIA 14 días,575
3,Huelva,Tasa PDIA 14 días,"112,58108285380318"
4,Huelva,Confirmados PDIA 7 días,289
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,31
630,Municipio de Huelva sin especificar,Curados,4


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  2049.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  548.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1178 personas en los últimos 7 días 

Un positivo PCR cada 756 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,2049.0,289.0,575.0,510743.0,56.584231,112.581083,83.0
Huelva-Costa,1058.0,178.0,303.0,288115.0,61.780886,105.166340,53.0
Huelva (capital),548.0,122.0,190.0,143663.0,84.920961,132.253955,34.0
Condado-Campiña,696.0,77.0,209.0,155057.0,49.659158,134.789142,19.0
Sierra de Huelva-Andévalo Central,264.0,32.0,55.0,67571.0,47.357594,81.395865,13.0
Granado (El),13.0,11.0,11.0,516.0,2131.782946,2131.782946,7.0
Lepe,147.0,16.0,44.0,27431.0,58.328169,160.402464,6.0
Palma del Condado (La),102.0,15.0,33.0,10761.0,139.392250,306.662950,5.0
Paterna del Campo,17.0,5.0,6.0,3509.0,142.490738,170.988886,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),13.0,11.0,11.0,516.0,2131.782946,2131.782946,7.0
Villarrasa,27.0,14.0,21.0,2176.0,643.382353,965.073529,2.0
San Juan del Puerto,74.0,7.0,53.0,9300.0,75.268817,569.892473,0.0
Jabugo,14.0,3.0,9.0,2250.0,133.333333,400.000000,0.0
Villanueva de los Castillejos,18.0,3.0,11.0,2820.0,106.382979,390.070922,0.0
Palma del Condado (La),102.0,15.0,33.0,10761.0,139.392250,306.662950,5.0
Zalamea la Real,28.0,4.0,7.0,3068.0,130.378096,228.161669,1.0
Bollullos Par del Condado,113.0,7.0,29.0,14272.0,49.047085,203.195067,1.0
Manzanilla,20.0,1.0,4.0,2135.0,46.838407,187.353630,0.0
